In [1]:
from dotenv import load_dotenv
import sys
sys.path.append("../../../")
from shared.models import MiniPileDataset
from shared.interp import count_non_zero_feature_activations, plot_feature_activation_histogram, plot_feature_activation_histogram_from_log_feature_densities, plot_highest_activating_feature_for_each_sentence, create_feature_heatmap_widget
from shared.features import Feature, FeatureSample
import os
import json

# Enable automatic reloading of modules when they change
%load_ext autoreload
%autoreload 2

/Users/josephtey/Projects/auto-ed-coder/venv/lib/python3.9/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_mutation' has been removed
  warnings.warn(message, UserWarning)


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch

MODEL_NAME_1 = "bert-base-uncased"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_1)
model = AutoModel.from_pretrained(MODEL_NAME_1)

# Encode some text
text = "This is a sample text to encode."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Move model to device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Get the embedding from the last hidden state
with torch.no_grad():
    outputs = model(**inputs)
embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

print("Embedding:", embedding)


Embedding: [ 1.84332848e-01 -1.71771586e-01  3.56589220e-02 -1.51276067e-01
 -1.85860202e-01 -4.03527260e-01  1.03313997e-01  2.31959239e-01
  1.80198148e-01  2.54761260e-02 -2.52800286e-01 -2.99641520e-01
 -3.12144727e-01  1.36302456e-01 -1.11550286e-01  1.23413272e-01
 -1.41808733e-01  1.18051521e-01  3.99289727e-02 -7.64348805e-02
  5.78769222e-02  1.11669153e-01 -5.93145967e-01  2.25300901e-02
  9.98427927e-01 -4.05542165e-01 -3.07221293e-01 -3.29373479e-01
 -8.23361218e-01 -8.15446228e-02  5.12531400e-03  2.83867598e-01
 -2.70921260e-01 -1.25358090e-01 -1.67635307e-01 -1.80304632e-01
  2.93781102e-01 -4.39690351e-02  3.00429743e-02  2.44957164e-01
 -5.87249517e-01 -4.26605135e-01  2.32217878e-01  1.81071863e-01
  5.76134287e-02 -3.51474196e-01 -5.17745763e-02 -2.05165073e-02
 -9.27073210e-02 -9.01999176e-02 -9.37584698e-01  4.22418475e-01
 -3.02045234e-02  5.15738368e-01  2.13608995e-01  3.28836083e-01
  2.70725578e-01 -9.43124831e-01 -1.13463707e-01 -1.97403207e-01
  5.24869144e-

In [ ]:
# Load the model from the pickle file
import pickle 
import sys

sys.path.append("../")
from shared.sparse_autoencoder import SparseAutoencoder, SparseAutoencoderConfig
import json

%load_ext autoreload
%autoreload 2

# load the dataset
pre_trained_sentences_file = "pre-trained/data/asap_sentences_3_bert-base-uncased.npy"
pre_trained_embeddings_file = "pre-trained/data/asap_embeddings_3_bert-base-uncased.npy"
pre_trained_mini_pile_dataset = MiniPileDataset(pre_trained_sentences_file, pre_trained_embeddings_file)

# Load the configuration from the JSON file
pre_trained_config_path = "pre-trained/sae/config.json"
with open(pre_trained_config_path, "r") as pre_trained_config_file:
    pre_trained_config = json.load(pre_trained_config_file)

# Load the pre-trained model from the pickle file
pre_trained_sae_config = SparseAutoencoderConfig(d_model=pre_trained_config["dimensions"], d_sparse=8 * pre_trained_config["dimensions"], sparsity_alpha=pre_trained_config["sparsity_alpha"])
pre_trained_model = SparseAutoencoder(pre_trained_sae_config)
pre_trained_model_path = "pre-trained/sae/sae.pkl"
with open(pre_trained_model_path, "rb") as pre_trained_f:
    pre_trained_model_state_dict = pickle.load(pre_trained_f)
    pre_trained_model.load_state_dict(pre_trained_model_state_dict)

# Load the log feature densities from the JSON file
pre_trained_log_feature_densities_path = "pre-trained/sae/log_feature_densities.json"
with open(pre_trained_log_feature_densities_path, "r") as pre_trained_json_file:
    pre_trained_log_feature_densities = json.load(pre_trained_json_file)

# Load features
pre_trained_features_folder = "pre-trained/features"
pre_trained_features = []
pre_trained_filtered_out_count = 0

for pre_trained_filename in os.listdir(pre_trained_features_folder):
    if pre_trained_filename.startswith("feature_") and pre_trained_filename.endswith(".json"):
        with open(os.path.join(pre_trained_features_folder, pre_trained_filename), "r") as pre_trained_json_file:
            pre_trained_feature_data = json.load(pre_trained_json_file)
            pre_trained_feature = Feature(**pre_trained_feature_data)
            pre_trained_features.append(pre_trained_feature)
            # if len(pre_trained_feature.high_act_samples) >= 10:
            #     pre_trained_features.append(pre_trained_feature)
            # else:
            #     pre_trained_filtered_out_count += 1

# Load the scores from the JSON file
with open("asap_sentences_scores_3.json", "r") as score_file:
    score_dict = json.load(score_file)

# Add scores to high acting samples in pre_trained_features
for feature in pre_trained_features:
    for sample in feature.high_act_samples:
        sample_text = sample.text
        if sample_text in score_dict:
            sample.score = score_dict[sample_text]
        else:
            sample.score = None  # or some default value if the text is not found

print("Scores added to high acting samples in pre_trained_features.")

# pre_trained_features.sort(key=lambda x: x.confidence, reverse=True)

# print(f"Number of filtered out samples: {pre_trained_filtered_out_count}")

In [3]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

class BottleneckT5Autoencoder:
    def __init__(self, model_path: str, device='cpu'):
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, model_max_length=512)
        self.model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to(self.device)
        self.model.eval()

    @torch.no_grad()
    def embed(self, text: str) -> torch.FloatTensor:
        inputs = self.tokenizer(text, return_tensors='pt').to(self.device)
        decoder_inputs = self.tokenizer('', return_tensors='pt').to(self.device)
        return self.model(
            **inputs,
            decoder_input_ids=decoder_inputs['input_ids'],
            encode_only=True,
        )[0]

    @torch.no_grad()
    def generate_from_latent(self, latent: torch.FloatTensor, max_length=512, temperature=1.0) -> str:
        dummy_text = '.'
        dummy = self.embed(dummy_text)
        perturb_vector = latent - dummy
        self.model.perturb_vector = perturb_vector
        input_ids = self.tokenizer(dummy_text, return_tensors='pt').to(self.device).input_ids
        output = self.model.generate(
            input_ids=input_ids,
            max_length=max_length,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            num_return_sequences=1,
        )
        return self.tokenizer.decode(output[0], skip_special_tokens=True)


In [8]:
!pip install sentencepiece


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/Users/josephtey/Projects/auto-ed-coder/venv/bin/python3.9 -m pip install --upgrade pip' command.


In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
autoencoder = BottleneckT5Autoencoder(model_path='thesephist/contra-bottleneck-t5-base-wikipedia', device=device)


ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.